In [22]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, LSTM, GRU, Flatten, TimeDistributed, SimpleRNN
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [23]:
dataset_path = '/content/drive/MyDrive/Datasets/joon_del/'
data_class_0 = os.path.join(dataset_path,'0')
data_class_1 = os.path.join(dataset_path,'1')

In [ ]:
%mkdir 'video_to_imgs'
%cd '/content/video_to_imgs/'
%mkdir '0'
%mkdir '1'

In [7]:
width = height = 70
batch_size = 64
epoches = 10
directory = '/content/video_to_imgs/'

In [31]:
def conver_to_dataset():

  frames = []
  labels = []
  
  # class 0
  for i in range(len(os.listdir(data_class_0))):
      video_cap = cv2.VideoCapture(f'{data_class_0}/0{i}.mp4')
      c = 0

      while True:
        success, frame = video_cap.read()
        if not success:
            break
        if c % 10 == 0:
          frame = cv2.resize(frame, dsize=(width, height)).astype('float16')
          frames.append(frame)
          labels.append(0)

        # cv2.imwrite(f"/content/video_to_imgs/0/{i}_{c}.jpg", frame)
        c += 1


  # class 1
  for i in range(len(os.listdir(data_class_1))):
      video_cap = cv2.VideoCapture(f'{data_class_1}/0{i}.mp4')
      c = 0

      while True:
        success, frame = video_cap.read()
        if not success:
            break

        if c % 10 == 0:
          frame = cv2.resize(frame, dsize=(width, height)).astype('float16')
          frames.append(frame)
          labels.append(1)

        # cv2.imwrite(f"/content/video_to_imgs/1/{i}_{c}.jpg", frame)
        # c += 1

  frames = np.array(frames)
  labels = np.array(labels)

  return frames, labels

In [32]:
X_data, Y_data = conver_to_dataset()

In [33]:
X_data.shape, Y_data.shape

((5357, 70, 70, 3), (5357,))

In [34]:
Y_data = Y_data[..., np.newaxis]
X_data = X_data[..., np.newaxis]

In [35]:
X_data.shape, Y_data.shape

((5357, 70, 70, 3, 1), (5357, 1))

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

In [37]:
model = tf.keras.models.Sequential([ 
                                   # CNN
                                   TimeDistributed(Conv2D(32, (3, 3), activation="relu",input_shape=(None, width, height, 3))), 
                                   TimeDistributed(MaxPooling2D(pool_size=(2, 2))), 
                                   TimeDistributed(Conv2D(16, (3, 3), activation="relu")),
                                   TimeDistributed(Flatten()), 
                                   # RNN 
                                   SimpleRNN(40), 
                                   Dense(2, activation="softmax") ])

In [38]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    metrics=["accuracy"]
)

In [39]:
model.fit(X_train, Y_train, batch_size = batch_size, epochs = epoches)

Epoch 1/10


ValueError: ignored

## *_*_*_*_*_*_*_*_*_*_*_*_ ALAKi *_*_*_*_*_*_*_*_*_*_*_*_

In [ ]:
data_generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2) # set validation split

train_generator = data_generator.flow_from_directory(
    directory,
    target_size = (height, width),
    batch_size = batch_size,
    shuffle = False,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = data_generator.flow_from_directory(
    directory, 
    target_size = (height, width),
    batch_size = batch_size,
    shuffle = False,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 8429 images belonging to 3 classes.
Found 2107 images belonging to 3 classes.


In [ ]:
'class 0: ', len(os.listdir('/content/video_to_imgs/0/')), 'class 1: ', len(os.listdir('/content/video_to_imgs/1/'))

('class 0: ', 5765, 'class 1: ', 4771)